In [34]:
import json
import pandas as pd

pd.options.display.max_rows = 999
# increase number of characters shown in one cell
pd.options.display.max_colwidth = 1000

In [2]:
# load data
with open("../songs_db.json") as f:
    songs_db = json.load(f)

print(f"Data was updated on {songs_db['latest_ts']}")
print("Dict keys=", list(songs_db.keys()))

Data was updated on 2023-12-17T11:45:31+0000
Dict keys= ['songs_dic', 'broken_ts_list', 'latest_ts']


In [25]:
df = (
    pd.DataFrame.from_dict(songs_db["songs_dic"], orient="index", columns=["song"])
    .reset_index()
    .rename(columns={"index": "ts"})
    .assign(ts=lambda x: pd.to_datetime(x["ts"]))
    .sort_values("ts")
    .reset_index(drop=True)
    .set_index("ts")
)

In [35]:
df.loc["2022-12-25":"2022-12-26"]

,song
ts,
2022-12-25 00:04:59+00:00,Seira Kagami - Super Special
2022-12-25 00:09:47+00:00,チト(CV.水瀬いのり)、ユーリ(CV.久保ユリカ) - 動く、動く
2022-12-25 00:12:47+00:00,ROUND TABLE feat. Nino - Groovin' Magic
2022-12-25 00:15:16+00:00,O-zone - Dragostea din tei
2022-12-25 00:18:53+00:00,Flow - Days
2022-12-25 00:23:10+00:00,FLOW - GO!!!
2022-12-25 00:24:44+00:00,Chiwa Saito - staple stable
2022-12-25 00:28:51+00:00,James Shimoji feat. Rob Laufer - REDLINE DAY
2022-12-25 00:32:37+00:00,Horie Yui - silky heart


In [42]:
df_christmas = []

for year in range(df.index.min().year, df.index.max().year + 1):
    df_christmas.append(df.loc[f"{year}-12-24" : f"{year}-12-26"])

df_christmas = pd.concat(df_christmas)

df_non_christmas = df.loc[~df.index.isin(df_christmas.index)]

In [63]:
normalized_christmas_songs = df_christmas.song.value_counts()
normalized_non_christmas_songs = df_non_christmas.song.value_counts()

chistmas_stats = (
    (
        normalized_christmas_songs.to_frame("christmas_plays").assign(
            christmas_play_rate=lambda x: x["christmas_plays"]
            / x["christmas_plays"].sum(),
            christmas_rank=lambda x: x["christmas_play_rate"].rank(ascending=False),
        )
    )
    .join(
        (
            normalized_non_christmas_songs.to_frame("non_christmas_plays").assign(
                non_christmas_play_rate=lambda x: x["non_christmas_plays"]
                / x["non_christmas_plays"].sum(),
                non_christmas_rank=lambda x: x["non_christmas_play_rate"].rank(
                    ascending=False
                ),
            )
        )
    )
    .fillna(0)
)

In [70]:
(
    chistmas_stats.sort_values("christmas_play_rate", ascending=False)
    .assign(
        christmas_excess=lambda x: (x["christmas_play_rate"] + 0.000165)
        / (x["non_christmas_play_rate"] + 0.000165),
        christmas_excess_v2=lambda x: (
            (x["christmas_plays"] + 1) / x.christmas_plays.sum()
        )
        / ((x["non_christmas_plays"] + 1) / x.non_christmas_plays.sum()),
    )
    .sort_values("christmas_excess", ascending=False)
    .loc[lambda x: x.non_christmas_plays > 1]
    .head(50)
)

,christmas_plays,christmas_play_rate,christmas_rank,non_christmas_plays,non_christmas_play_rate,non_christmas_rank,christmas_excess,christmas_excess_v2
song,,,,,,,,
Shogo Hamada - MIDNIGHT FLIGHT - Hitoribocchi no Christmas Eve,13,0.001074,18.0,5.0,0.000003,26447.0,7.354230,97.400958
"Eriko Nakamura (中村 繪里子), Asami Imai (今井 麻美), Mayako Nigo (仁後 真耶子), Hiromi Hirata (平田 宏美) & Akiko Hasegawa (長谷川 明子) - Merry",13,0.001074,18.0,19.0,0.000013,16180.5,6.954962,29.220287
Hisakawa Aya - Jingle Bells,17,0.001404,8.5,108.0,0.000075,2160.0,6.549475,6.893384
Ito Chieri - Merry Christmas,14,0.001156,14.0,70.0,0.000048,7128.0,6.193742,8.819000
Rie Kugimiya - Holy Night,18,0.001487,6.5,177.0,0.000122,622.0,5.750358,4.455742
"Ryuichi Sakamoto - Ryuichi Sakamoto - Merry Christmas, Mr. Lawrence (Piano Version)",10,0.000826,35.5,19.0,0.000013,16180.5,5.563737,22.958797
Tokyo Brass Style - Brasta Christmas Sweets,8,0.000661,70.0,5.0,0.000003,26447.0,4.902410,62.614902
歌組雪月花 - 回レ！雪月花,14,0.001156,14.0,178.0,0.000123,612.5,4.589107,3.498039
Hyadain - Christmas? Sore Nani? Oishii no? 2011Ver.,7,0.000578,100.0,2.0,0.000001,48206.5,4.466997,111.315381


In [66]:
0.000578 / 7

8.257142857142857e-05